In [ ]:
# Improved version with better error handling and fallback options
import requests
import base64
import json
import os
import time

# Your OpenRouter API key
API_KEY = ""

# Models to try
VISION_MODEL = "meta-llama/llama-3.2-11b-vision-instruct"
TEXT_MODEL = "meta-llama/llama-3.1-8b-instruct:free"

def encode_image(image_path):
    """Encode image to base64"""
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except Exception as e:
        print(f"Error encoding {image_path}: {e}")
        return None

def get_single_image_description(image_path, max_retries=3):
    """Get description for a single image with retry logic"""
    for attempt in range(max_retries):
        try:
            # print(f"  Attempt {attempt + 1} for {os.path.basename(image_path)}")
            
            # Encode the image
            base64_image = encode_image(image_path)
            if not base64_image:
                return f"Failed to encode image: {os.path.basename(image_path)}"
            
            # Determine mime type
            filename = os.path.basename(image_path).lower()
            mime_type = "image/png" if filename.endswith(".png") else "image/jpeg"

            # API endpoint
            url = "https://openrouter.ai/api/v1/chat/completions"

            # Headers
            headers = {
                "Authorization": f"Bearer {API_KEY}",
                "Content-Type": "application/json"
            }

            # Payload for single image
            payload = {
                "model": VISION_MODEL,
                "messages": [
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": """Describe this apartment space in detail, focusing on features like rooms, swimming pools, balcony, attached bathrooms, and other amenities visible in the image. Provide a concise yet comprehensive description suitable for a rental listing.
"""
                            },
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:{mime_type};base64,{base64_image}"
                                }
                            }
                        ]
                    }
                ],
                "max_tokens": 500,
                "temperature": 0.7
            }

            # Send request
            response = requests.post(url, headers=headers, data=json.dumps(payload), timeout=30)
            
            if response.status_code == 200:
                result = response.json()
                if 'choices' in result and len(result['choices']) > 0:
                    content = result['choices'][0]['message']['content']
                    # Filter out suspicious responses
                    if len(content) > 20 and not content.startswith("safesafesafe"):
                        return content
                    else:
                        print(f"    Suspicious response, retrying...")
                        continue
                else:
                    print(f"    No choices in response: {result}")
                    if attempt == max_retries - 1:
                        return f"No valid response for {os.path.basename(image_path)}"
            else:
                print(f"    HTTP {response.status_code}: {response.text}")
                if attempt == max_retries - 1:
                    return f"API Error for {os.path.basename(image_path)}: {response.status_code}"
            
            # Wait before retry
            if attempt < max_retries - 1:
                time.sleep(2)
                
        except Exception as e:
            print(f"    Exception: {e}")
            if attempt == max_retries - 1:
                return f"Error processing {os.path.basename(image_path)}: {str(e)}"
            time.sleep(2)
    
    return f"Failed to process {os.path.basename(image_path)} after {max_retries} attempts"

def create_natural_combined_description(descriptions):
    """Create a grammatically correct, first-person description"""
    try:
        # Filter valid descriptions
        valid_descriptions = []
        for desc in descriptions:
            if not any(error_phrase in desc.lower() for error_phrase in 
                      ['error', 'failed', 'safesafesafe', 'suspicious response']):
                valid_descriptions.append(desc)
        
        if not valid_descriptions:
            return "I'm unable to generate a description from the provided images."
        
        # Combine all descriptions
        full_text = ' '.join(valid_descriptions)
        text_lower = full_text.lower()
        
        # Extract key information intelligently
        rooms_info = {}
        features_info = []
        amenities_info = []
        
        # Extract room details with context
        if 'kitchen' in text_lower:
            kitchen_features = []
            if 'granite' in text_lower: kitchen_features.append('granite countertops')
            if 'stainless steel' in text_lower: kitchen_features.append('stainless steel appliances')
            if 'modern' in text_lower and 'kitchen' in text_lower: kitchen_features.append('modern design')
            if 'spacious' in text_lower and 'kitchen' in text_lower: kitchen_features.append('spacious layout')
            rooms_info['kitchen'] = kitchen_features
        
        if 'bathroom' in text_lower:
            bathroom_features = []
            if 'spa' in text_lower or 'luxury' in text_lower: bathroom_features.append('spa-like amenities')
            if 'tub' in text_lower or 'bathtub' in text_lower: bathroom_features.append('soaking tub')
            if 'shower' in text_lower: bathroom_features.append('shower')
            if 'tile' in text_lower: bathroom_features.append('tile finishes')
            rooms_info['bathroom'] = bathroom_features
        
        if 'living' in text_lower or 'bedroom' in text_lower:
            living_features = []
            if 'spacious' in text_lower: living_features.append('spacious')
            if 'bright' in text_lower or 'natural light' in text_lower: living_features.append('plenty of natural light')
            if 'open' in text_lower: living_features.append('open layout')
            rooms_info['living'] = living_features
        
        # Extract amenities
        if 'pool' in text_lower or 'swimming' in text_lower:
            amenities_info.append('swimming pool')
        if 'gym' in text_lower or 'fitness' in text_lower:
            amenities_info.append('fitness center')
        if 'parking' in text_lower:
            amenities_info.append('parking')
        if 'balcony' in text_lower:
            amenities_info.append('private balcony')
        
        # Extract overall features
        if 'modern' in text_lower:
            features_info.append('modern')
        if 'spacious' in text_lower:
            features_info.append('spacious')
        if 'updated' in text_lower:
            features_info.append('updated')
        if 'luxury' in text_lower:
            features_info.append('luxury')
        
        # Build grammatically correct description
        description_parts = []
        
        # Dynamic opening sentences based on actual features found
        dynamic_openers = []
        
        # Feature-based openers
        if 'luxury' in features_info:
            dynamic_openers.extend([
                "I'm thrilled to share my luxury apartment with discerning renters",
                "My luxury apartment offers an elevated living experience",
                "Experience upscale living in my luxury apartment",
                "I'm excited to offer this premium luxury space"
            ])
        elif 'modern' in features_info:
            dynamic_openers.extend([
                "I'm proud to present my modern apartment for sublease",
                "My contemporary apartment combines style with functionality",
                "Step into modern living with my updated apartment",
                "I'd love to share my sleek, modern space with you"
            ])
        elif 'spacious' in features_info:
            dynamic_openers.extend([
                "My spacious apartment offers room to breathe and relax",
                "Enjoy generous living space in my roomy apartment",
                "I'm excited to offer my spacious home for sublease",
                "My apartment provides the space you've been looking for"
            ])
        elif 'updated' in features_info:
            dynamic_openers.extend([
                "My recently updated apartment is move-in ready",
                "I'm pleased to offer my refreshed and updated space",
                "Experience the comfort of my newly updated apartment"
            ])
        
        # Amenity-based openers
        if amenities_info:
            if 'swimming pool' in amenities_info and 'fitness center' in amenities_info:
                dynamic_openers.extend([
                    "My apartment comes with resort-style amenities you'll love",
                    "I'm offering my apartment with fantastic recreational facilities",
                    "Enjoy apartment living with premium amenities included"
                ])
            elif 'swimming pool' in amenities_info:
                dynamic_openers.extend([
                    "My apartment includes access to a beautiful swimming pool",
                    "Cool off anytime with pool access from my apartment"
                ])
            elif 'fitness center' in amenities_info:
                dynamic_openers.extend([
                    "Stay fit with gym access included in my apartment",
                    "My apartment comes with convenient fitness facilities"
                ])
        
        # Room combination openers
        kitchen_and_living = 'kitchen' in rooms_info and 'living' in rooms_info
        all_rooms = len(rooms_info) >= 3
        
        if all_rooms:
            dynamic_openers.extend([
                "I'm excited to share my complete apartment with everything you need",
                "My full apartment setup is perfect for comfortable living",
                "Experience a well-appointed home in my apartment"
            ])
        elif kitchen_and_living:
            dynamic_openers.extend([
                "My apartment offers great spaces for cooking and relaxing",
                "Enjoy the perfect combination of kitchen and living space"
            ])
        
        # Fallback general openers with personality
        if not dynamic_openers:
            dynamic_openers = [
                "I'm delighted to offer my apartment for sublease",
                "My apartment is ready for someone special to call home",
                "I'd love to welcome you to my comfortable apartment",
                "Experience quality living in my well-maintained apartment",
                "My apartment offers exactly what you're looking for",
                "I'm confident my apartment will exceed your expectations",
                "Discover your next home in my inviting apartment",
                "My apartment provides the perfect living solution",
                "I'm excited to share my beautiful apartment with you"
            ]
        
        # Select opener based on content hash for consistency but variety
        content_hash = hash(full_text[:100] + str(len(amenities_info)) + str(len(rooms_info)))
        selected_opener = dynamic_openers[content_hash % len(dynamic_openers)]
        description_parts.append(selected_opener)
        
        # Room descriptions with proper grammar
        room_sentences = []
        
        if 'kitchen' in rooms_info and rooms_info['kitchen']:
            kitchen_desc = "The kitchen features " + ', '.join(rooms_info['kitchen'][:-1])
            if len(rooms_info['kitchen']) > 1:
                kitchen_desc += f", and {rooms_info['kitchen'][-1]}"
            else:
                kitchen_desc = f"The kitchen offers {rooms_info['kitchen'][0]}"
            room_sentences.append(kitchen_desc)
        elif 'kitchen' in rooms_info:
            room_sentences.append("The kitchen is well-appointed for all your cooking needs")
        
        if 'bathroom' in rooms_info and rooms_info['bathroom']:
            bathroom_desc = "The bathroom includes " + ', '.join(rooms_info['bathroom'][:-1])
            if len(rooms_info['bathroom']) > 1:
                bathroom_desc += f", and {rooms_info['bathroom'][-1]}"
            else:
                bathroom_desc = f"The bathroom features {rooms_info['bathroom'][0]}"
            room_sentences.append(bathroom_desc)
        elif 'bathroom' in rooms_info:
            room_sentences.append("The bathroom is clean and well-maintained")
        
        if 'living' in rooms_info and rooms_info['living']:
            living_desc = "The living space offers " + ' and '.join(rooms_info['living'])
            room_sentences.append(living_desc)
        elif 'living' in rooms_info:
            room_sentences.append("The living area provides comfortable space for relaxation")
        
        # Add room descriptions
        if room_sentences:
            if len(room_sentences) == 1:
                description_parts.append(room_sentences[0])
            elif len(room_sentences) == 2:
                description_parts.append(f"{room_sentences[0]}, while {room_sentences[1].lower()}")
            else:
                combined_rooms = f"{room_sentences[0]}, {room_sentences[1].lower()}, and {room_sentences[2].lower()}"
                description_parts.append(combined_rooms)
        
        # Add amenities with proper grammar
        if amenities_info:
            if len(amenities_info) == 1:
                amenity_sentence = f"You'll have access to the {amenities_info[0]}"
            elif len(amenities_info) == 2:
                amenity_sentence = f"You'll have access to the {amenities_info[0]} and {amenities_info[1]}"
            else:
                amenity_sentence = f"You'll have access to the {', '.join(amenities_info[:-1])}, and {amenities_info[-1]}"
            description_parts.append(amenity_sentence)
        
        # Personal closing with confidence
        if amenities_info and len(rooms_info) >= 2:
            closing = "With all these amenities and thoughtful design, I believe you'll find this to be the perfect home"
        elif 'modern' in features_info:
            closing = "The modern touches throughout make this a truly special place to live"
        elif 'spacious' in features_info:
            closing = "The generous space and layout create an ideal living environment"
        else:
            closing = "I'm confident this apartment will meet all your needs"
        
        description_parts.append(closing)
        
        # Call to action
        action_phrases = [
            "Available for immediate sublease - contact me to schedule a viewing",
            "Ready for move-in - I'd love to show you around",
            "Contact me today to make this your new home"
        ]
        
        action = action_phrases[hash(''.join(description_parts)) % len(action_phrases)]
        description_parts.append(action)
        
        # Create final description with proper punctuation
        final_description = '. '.join(description_parts)
        if not final_description.endswith('.'):
            final_description += '.'
            
        return final_description
        
    except Exception as e:
        return f"I apologize, but there was an issue creating the apartment description: {str(e)}"

def process_apartment_images(image_dir):
    """Main function to process all images in directory"""
    try:
        # Supported image extensions
        valid_extensions = (".jpg", ".jpeg", ".png")
        descriptions = []
        
        print("Processing apartment images...\n")
        
        # Get all image files
        image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(valid_extensions)]
        
        if not image_files:
            return "No valid images found in the directory."
        
        print(f"Found {len(image_files)} image(s): {', '.join(image_files)}\n")
        
        # Process each image and show individual descriptions
        for i, filename in enumerate(image_files, 1):
            image_path = os.path.join(image_dir, filename)
            print(f"📷 IMAGE {i}: {filename}")
            print("-" * 50)
            
            description = get_single_image_description(image_path)
            descriptions.append(description)
            
            # Show the individual description
            print(f"DESCRIPTION: {description}\n")
        
        # Create combined description
        print("=" * 70)
        print("CREATING FINAL APARTMENT DESCRIPTION...")
        print("=" * 70)
        combined_description = create_natural_combined_description(descriptions)
        
        return combined_description
        
    except Exception as e:
        return f"Error processing apartment images: {str(e)}"

# Run the improved version
if __name__ == "__main__":
    image_dir = r"C:\Users\MSI\Downloads\webscrap\images"
    
    print("APARTMENT IMAGE ANALYZER")
    print("=" * 50)
    print(f"Processing images from: {image_dir}\n")
    
    final_description = process_apartment_images(image_dir)
    
    print("\n" + "=" * 60)
    print("FINAL APARTMENT DESCRIPTION:")
    print("=" * 60)
    print(final_description)

APARTMENT IMAGE ANALYZER
Processing images from: C:\Users\MSI\Downloads\webscrap\images

Processing apartment images...

Found 3 image(s): 2.jpg, 3.jpg, 4.png

📷 IMAGE 1: 2.jpg
--------------------------------------------------
DESCRIPTION: The image depicts an apartment complex with a pool and lounge chairs in the foreground, surrounded by trees and buildings in the background.

* **Pool Area:**
	+ Rectangular pool with blue tiles around the edges.
	+ Two hot tubs, one at each end of the pool.
	+ Ladder entrance on the left side of the pool.
	+ Lounge chairs and umbrellas around the pool.
	+ Trees planted in the ground.
* **Buildings:**
	+ Multiple buildings with balconies and windows.
	+ Buildings are white or gray with dark accents.
	+ Balconies have black railings.
	+ Windows are rectangular with white frames.
* **Sky:**
	+ Blue sky with a few clouds.

The overall atmosphere of the image is one of relaxation and luxury, with the pool and lounge chairs suggesting a comfortable and e